In [1]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [2]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import handshape_datasets as hd
from IPython import display
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, LeakyReLU, Dropout, Flatten
from tensorflow.keras import Model

In [3]:
#Check GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
print(tf.config.experimental.list_logical_devices('GPU'))
tf.test.is_gpu_available()

1 Physical GPUs, 1 Logical GPUs
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
data = hd.load('PugeaultASL_A')

good_min = 40
good_classes = []
n_unique = len(np.unique(data[1]['y']))
for i in range(n_unique):
    images = data[0][np.equal(i, data[1]['y'])]
    if len(images) >= good_min:
        good_classes = good_classes + [i]
        
x = data[0][np.in1d(data[1]['y'], good_classes)]
img_shape = x[0].shape

y = data[1]['y'][np.in1d(data[1]['y'], good_classes)]
y_dict = dict(zip(np.unique(y), range(len(np.unique(y)))))
y = np.vectorize(y_dict.get)(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2, stratify=y)

classes = np.unique(y_train)
n_classes = len(classes)

INFO:Loading PugeaultASL_A...


In [17]:
def model():
    model = tf.keras.Sequential()
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same',input_shape=[32, 32, 3]))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(n_classes, activation='softmax'))

    return model

In [6]:
model = model()
model.summary()
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 64)        4864      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0

In [8]:
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
412/412 [==============================] - 3s 8ms/step - loss: 2.9505 - accuracy: 0.6072 - val_loss: 0.8413 - val_accuracy: 0.7976
Epoch 2/20
412/412 [==============================] - 3s 8ms/step - loss: 0.8101 - accuracy: 0.8098 - val_loss: 0.7249 - val_accuracy: 0.8369
Epoch 3/20
412/412 [==============================] - 3s 8ms/step - loss: 0.8255 - accuracy: 0.8333 - val_loss: 0.6582 - val_accuracy: 0.8630
Epoch 4/20
412/412 [==============================] - 3s 8ms/step - loss: 1.2969 - accuracy: 0.8193 - val_loss: 1.3534 - val_accuracy: 0.8552
Epoch 5/20
412/412 [==============================] - 3s 8ms/step - loss: 2.8113 - accuracy: 0.8001 - val_loss: 2.8280 - val_accuracy: 0.8319
Epoch 6/20
412/412 [==============================] - 3s 8ms/step - loss: 3.2950 - accuracy: 0.8320 - val_loss: 3.7167 - val_accuracy: 0.8401
Epoch 7/20
412/412 [==============================] - 3s 8ms/step - loss: 3.4561 - accuracy: 0.8510 - val_loss: 2.6499 - val_accuracy: 0.8721
Epoch 

In [9]:
data = hd.load('PugeaultASL_A')

good_min = 40
good_classes = []
n_unique = len(np.unique(data[1]['y']))
for i in range(n_unique):
    images = data[0][np.equal(i, data[1]['y'])]
    if len(images) >= good_min:
        good_classes = good_classes + [i]
        
x = data[0][np.in1d(data[1]['y'], good_classes)]
img_shape = x[0].shape

y = data[1]['y'][np.in1d(data[1]['y'], good_classes)]
y_dict = dict(zip(np.unique(y), range(len(np.unique(y)))))
y = np.vectorize(y_dict.get)(y)

# shapes
img_n , height, weight, chanels = x.shape

#Normalization
x = x.reshape(img_n, height, weight, chanels).astype('float32')
x = (x - 127.5) / 127.5  # Normalize the images to [-1, 1]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2, stratify=y)

classes = np.unique(y_train)
n_classes = len(classes)

#x_tuple = list(zip(x_train_aug,y_train))
#x_train_aug

print(x_train.shape, y_train.shape)

INFO:Loading PugeaultASL_A...


(52619, 32, 32, 3) (52619,)


In [10]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(int(height/4)*int(weight/4)*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((int(height/4), int(weight/4), 256)))

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    
    return model

In [11]:
generator = make_generator_model()
generator.summary()
num_examples_to_generate = 3000
noise = tf.random.normal([num_examples_to_generate, 100])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16384)             1638400   
_________________________________________________________________
batch_normalization (BatchNo (None, 16384)             65536     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16384)             0         
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         819200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 128)         512       
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 8, 8, 128)        

In [12]:

weights_path = '../GANs/Basic-GAN/results/generators_weights/PugeaultASL_A/class_{}/generator.h5'
for i in range(16):
        generator.load_weights(weights_path.format(i))
        generated_image = generator(noise, training=False).numpy()
        x_train = np.concatenate((x_train, generated_image), axis=0)
        y_tmp = np.ones(num_examples_to_generate,)* i
        y_train = np.concatenate((y_train,y_tmp), axis=0)
        
print(x_train.shape, y_train.shape)


(100619, 32, 32, 3) (100619,)


In [13]:
shuffler = np.random.permutation(x_train.shape[0])
x_train = x_train[shuffler]
y_train = y_train[shuffler]
print(x_train.shape, y_train.shape)

(100619, 32, 32, 3) (100619,)


In [18]:
model2 = model()
model2.summary()
model2.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 16, 16, 64)        4864      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)             

In [19]:
model2.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
787/787 [==============================] - 7s 9ms/step - loss: 0.4730 - accuracy: 0.8604 - val_loss: 0.3396 - val_accuracy: 0.8968
Epoch 2/20
787/787 [==============================] - 6s 8ms/step - loss: 0.1685 - accuracy: 0.9479 - val_loss: 0.2536 - val_accuracy: 0.9216
Epoch 3/20
787/787 [==============================] - 6s 8ms/step - loss: 0.1252 - accuracy: 0.9607 - val_loss: 0.1718 - val_accuracy: 0.9504
Epoch 4/20
787/787 [==============================] - 6s 8ms/step - loss: 0.1038 - accuracy: 0.9673 - val_loss: 0.1474 - val_accuracy: 0.9551
Epoch 5/20
787/787 [==============================] - 6s 8ms/step - loss: 0.0908 - accuracy: 0.9714 - val_loss: 0.1414 - val_accuracy: 0.9605
Epoch 6/20
787/787 [==============================] - 6s 8ms/step - loss: 0.0891 - accuracy: 0.9723 - val_loss: 0.1432 - val_accuracy: 0.9578
Epoch 7/20
787/787 [==============================] - 6s 8ms/step - loss: 0.0784 - accuracy: 0.9753 - val_loss: 0.3215 - val_accuracy: 0.9254
Epoch 